In [1]:
import threading
# Helper thread to avoid the Spark StreamingContext from blocking Jupyter
class StreamingThread(threading.Thread):
    def __init__(self, ssc):
        super().__init__()
        self.ssc = ssc
    def run(self):
        self.ssc.start()
        self.ssc.awaitTermination()
    def stop(self):
        print('----- Stopping... this may take a few seconds -----')
        self.ssc.stop(stopSparkContext=False, stopGraceFully=True)

In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [3]:
import os
from pyspark.streaming import StreamingContext

In [4]:
# Save less frequently and create less subdirectories. I.e. every 120 seconds
ssc = StreamingContext(sc, 120)

C:\Users\Dillon\Desktop\spark\spark-3.5.1-bin-hadoop3\python\pyspark\streaming\context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(


In [5]:
lines = ssc.socketTextStream("seppe.net", 7778)

In [6]:
out_dir = f"{os.path.abspath('')}{os.path.sep}saved_stories"
lines.saveAsTextFiles(f"file:///{out_dir}")
print("Saving to", out_dir)

Saving to C:\Users\Dillon\Desktop\spark\notebooks\saved_stories


In [7]:
ssc_t = StreamingThread(ssc)
ssc_t.start()

In [ ]:
# Don't run this cell unless you want to stop. You should see subdirectories appear in the out_dir
ssc_t.stop()

----- Stopping... this may take a few seconds -----


In [ ]:
import json
# Function to read and parse specific files from a directory and its subdirectories
def read_text_files(base_directory):
    data = []
    for root, dirs, files in os.walk(base_directory):
        for file in files:
            if file.startswith("part"):# Check if the file name starts with "part"
                file_path = os.path.join(root, file)
                with open(file_path, "r", encoding="utf-8") as f:
                    content = f.read().strip()# Read the entire content of the file
                    # Split content by newlines to handle multiple JSON objects
                    json_objects = content.split('\n')
                    for json_obj in json_objects:
                        if json_obj.strip():# Check if the line is not empty
                            try:
                                line_data = json.loads(json_obj)
                                data.append(line_data)# Append the dictionary to the data list
                            except json.JSONDecodeError as e:
                                print(f"Error decoding JSON from file {file_path}: {e}")
                                continue
    return pd.DataFrame(data)

# Base directory where the text files are located 
base_directory = "C:\\Users\\Dillon\\Desktop\\spark\\notebooks\\saved_stories" #### YOUR DIRECTORY HERE ####
# Read the text files and store in a DataFrame
raw_df = read_text_files(base_directory)
# Save the DataFrame to a csv file
raw_df.to_csv("assignment3_DIRTY.csv", index=False)